In [1]:
!pip install cvxpylayers

In [2]:
import torch
import numpy as np
# import diffcp
import time
from dataclasses import dataclass
from typing import Any
import cvxpy as cp
import tracemalloc
import os
import linecache
from scipy.linalg import block_diag
from cvxpylayers.torch import CvxpyLayer
import cvxpy as cp

In [3]:
n = 100
n_eq_constraints   = 0
n_ineq_constraints = 50
Q = torch.eye(n)
q = torch.rand(n)
q.requires_grad_(True)
A = torch.empty(n_eq_constraints, n)
b = torch.empty(n_eq_constraints)
G = torch.randn(n_ineq_constraints, n)
h = torch.randn(n_ineq_constraints)

optimizer = torch.optim.SGD([q], lr=0.1)

Q_cp = cp.Parameter((n, n), PSD=True)
q_cp = cp.Parameter(n)
# A_cp = cp.Parameter((n_eq_constraints, n))
# b_cp = cp.Parameter(n_eq_constraints)
G_cp = cp.Parameter((n_ineq_constraints, n))
h_cp = cp.Parameter(n_ineq_constraints)
z_cp = cp.Variable(n)

objective_fn = 0.5 * cp.sum_squares(Q_cp @ z_cp) + q_cp.T @ z_cp
constraints = [G_cp @ z_cp <= h_cp, cp.SOC(1.0, z_cp)]

print('calling cvxpylaer...')
problem = cp.Problem(cp.Minimize(objective_fn), constraints)

layer = CvxpyLayer(problem, parameters=[Q_cp, q_cp, G_cp, h_cp], variables=[z_cp])
sol = layer(Q, q, G, h)
z_sol = sol[0]

loss = torch.sum(z_sol)
loss.backward()

cvxpylayer_grad = q.grad.clone().detach()
optimizer.zero_grad()


calling cvxpylaer...


/usr/local/lib/python3.12/dist-packages/cvxpy/reductions/solvers/solving_chain.py:250: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


In [4]:
cvxpylayer_grad

tensor([-0.0927, -0.0670,  0.0537,  0.0346, -0.0355, -0.0399, -0.0991, -0.0014,
         0.0337, -0.1033, -0.1005, -0.0450, -0.0436,  0.0510,  0.0418,  0.0267,
         0.0004,  0.0900,  0.0181, -0.0024, -0.0353, -0.0713, -0.0380, -0.0057,
         0.0374, -0.0835,  0.0069,  0.0585, -0.1341, -0.0837, -0.0828, -0.1245,
         0.0007, -0.1392, -0.1034,  0.0607,  0.0690, -0.0878, -0.1030, -0.1547,
         0.0017,  0.0036, -0.0331, -0.0291, -0.0230,  0.0922,  0.0536,  0.0317,
         0.0158, -0.0318, -0.0032,  0.0004, -0.1250, -0.0825, -0.0931,  0.0254,
        -0.0060,  0.0626, -0.0911, -0.1390, -0.1242, -0.0931, -0.0537, -0.0521,
        -0.0415, -0.1015, -0.1038,  0.0045, -0.1032, -0.0645,  0.0192, -0.1169,
        -0.0199, -0.0933,  0.0826, -0.0353,  0.0639,  0.0258, -0.0720,  0.0008,
        -0.0075, -0.0290,  0.1045,  0.1128,  0.0368, -0.0493, -0.0086, -0.2030,
         0.0668,  0.0022, -0.0456, -0.0586, -0.0605, -0.0515, -0.0727, -0.0932,
        -0.1083, -0.0438, -0.0422,  0.01

In [5]:
def forward_single_qsocp(Q, p, G, h, A, b, l2reg=0.0, l2con=True, verbose=False):
    nz, neq, nineq = p.shape[0], A.shape[0] if A is not None else 0, G.shape[0] if G is not None else 0

    z_ = cp.Variable(nz)

    obj = cp.Minimize(
        0.5 * cp.quad_form(z_, Q) + p.T @ z_ + cp.pnorm(z_, p=2) * l2reg)
    eqCon = A @ z_ == b if neq > 0 else None
    if nineq > 0:
        slacks = cp.Variable(nineq)
        ineqCon = G @ z_ + slacks == h
        slacksCon = slacks >= 0
    else:
        ineqCon = slacks = slacksCon = None
    if l2con:
      socCon = cp.SOC(1.0, z_)
    else:
      socCon = None
    cons = [x for x in [eqCon, ineqCon, slacksCon, socCon] if x is not None]
    prob = cp.Problem(obj, cons)
    prob.solve(verbose=verbose) # max_iters=5000)
    # prob.solve()
    # prob.solve(adaptive_rho = False)  # solver=cp.SCS, max_iters=5000, verbose=False)
    # prob.solve(solver=cp.SCS, max_iters=10000, verbose=True)
    assert('optimal' in prob.status)
    zhat = np.array(z_.value).ravel()
    nu = np.array(eqCon.dual_value).ravel() if eqCon is not None else None
    if ineqCon is not None:
        lam = np.array(ineqCon.dual_value).ravel()
        slacks = np.array(slacks.value).ravel()
    else:
        lam = slacks = None
    cdual = socCon.dual_value if socCon is not None else None
    return prob.value, zhat, nu, lam, slacks, cdual

In [6]:
p = q.detach()
_, zhati, nui, lami, si, cdual = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, p, G, h, None, None)]
)

In [7]:
np.linalg.norm(zhati - z_sol.detach().cpu().numpy())

np.float64(5.9796013687570054e-05)

In [8]:
torch.norm(Q @ zhati + p + G.T @ lami - cdual[1][:, 0])

/tmp/ipython-input-141708208.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  torch.norm(Q @ zhati + p + G.T @ lami - cdual[1][:, 0])


tensor(4.5767e-10, dtype=torch.float64)

In [9]:
c = torch.from_numpy(cdual[1]).T
d = torch.from_numpy(cdual[0])

dual_cutoff = 1e-4
active = (torch.from_numpy(lami) > dual_cutoff).float()
G_active = torch.cat((G * active.unsqueeze(-1), c), dim=0)
h_active = torch.cat((h * active, -d), dim=0)
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, p, None, None, G_active, h_active)], l2con=False, l2reg=float(d)
)[1]
np.linalg.norm(newzhat - zhati)

np.float64(3.686222212547046e-05)

In [10]:
alpha = 1
newp = p + torch.ones_like(p) / alpha
# theor. correct ffoqp
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, None, None, G_active, h_active)], l2con=False, l2reg=float(d)
)[1]
ffoqp_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(ffoqp_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(ffoqp_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(1.6733, dtype=torch.float64)
cosine similarity tensor(1.0000, dtype=torch.float64)


In [11]:
# lpgd
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, G, h, None, None)]
)[1]
lpgd_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(lpgd_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(lpgd_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(3.6625, dtype=torch.float64)
cosine similarity tensor(0.9838, dtype=torch.float64)


In [12]:
# ffoqp linCon only
newzhat = forward_single_qsocp(
    *[x.cpu().numpy() if x is not None else None
      for x in (Q, newp, None, None, G_active[:-1], h_active[:-1])]
)[1]
ffoqp_grad = torch.from_numpy((newzhat - zhati) * alpha)
print('gradient difference', torch.norm(ffoqp_grad - cvxpylayer_grad, p=1))
print('cosine similarity', torch.nn.functional.cosine_similarity(ffoqp_grad, cvxpylayer_grad, dim=0))


gradient difference tensor(3.6165, dtype=torch.float64)
cosine similarity tensor(0.9873, dtype=torch.float64)
